# 1. Préparation

In [1]:
import pandas as pd
import pulp
import numpy as np
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment
import geopandas as gpd
from shapely.geometry import Point
from pyproj import Proj, transform


In [2]:

import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment

# 데이터 로드

data = pd.read_csv('최종레이블 유동견인 등급 및 UTM좌표 포함.csv')


In [3]:
data.head()

,Unnamed: 0,id,행정동,centroid_x,centroid_y,클러스터링,유동인구,견인횟수,유동인구_레이블,견인횟수_레이블,score,유동인구+견인횟수
0,0,1575,화양동,963184.792317,1.949185e+06,0,323.728395,1.0,2,1,130.091358,7.0
1,1,1575,화양동,963184.792317,1.949185e+06,0,323.728395,1.0,2,1,130.091358,4.0
2,2,1575,화양동,963184.792317,1.949185e+06,0,323.728395,1.0,2,1,130.091358,2.0
3,3,1404,화양동,962924.984696,1.949235e+06,0,0.000000,0.0,2,1,0.000000,7.0
4,4,1405,화양동,962924.984696,1.949135e+06,0,0.000000,0.0,2,1,0.000000,7.0


In [4]:


# 각 레이블 조합과 클러스터링 별로 개수 집계
count_per_label_cluster = data.groupby(['유동인구+견인횟수','클러스터링']).size().reset_index(name='개수')
count_per_label = data.groupby(['유동인구+견인횟수']).size().reset_index(name='개수')

print('레이블 조합별 클러스터별 헥사곤 개수')
print(count_per_label_cluster)
print('')
print('레이블 조합별 클러스터별 헥사곤 개수')
print(count_per_label)

레이블 조합별 클러스터별 헥사곤 개수
    유동인구+견인횟수  클러스터링   개수
0         1.0      0  130
1         1.0      1  237
2         1.0      2  103
3         1.0      3   30
4         2.0      0  134
5         2.0      1  148
6         2.0      2   21
7         2.0      3  125
8         3.0      0   45
9         3.0      1   26
10        3.0      2   14
11        4.0      0  358
12        4.0      1  263
13        4.0      2   45
14        4.0      3   22
15        6.0      0    8
16        6.0      1   56
17        6.0      2   11
18        7.0      0  190
19        7.0      1   25
20        7.0      2   39
21        7.0      3    9
22        8.0      0   38
23        8.0      1    6
24        8.0      2    6
25        8.0      3    1

레이블 조합별 클러스터별 헥사곤 개수
   유동인구+견인횟수   개수
0        1.0  500
1        2.0  428
2        3.0   85
3        4.0  688
4        6.0   75
5        7.0  263
6        8.0   51


# Analysis

## 주차장 할당 최적화 모델 구축

In [5]:
'''
# 문제 인스턴스 생성
model = pulp.LpProblem("Enhanced_Parking_Allocation", pulp.LpMinimize)

# 클러스터 및 레이블 조합별 변수 생성
parking_vars = pulp.LpVariable.dicts("Park",
                                     [(i, j) for i in data['클러스터링'].unique() for j in range(4)],
                                     lowBound=0, cat='Integer')

# 목표 함수 설정: 주차장 수 최소화, 레이블 가중치 반영
    #(4-j)의 경우 레이블 등급에 따른 가중치 부여하는 과정(0등급일수록 높은 등급)
model += pulp.lpSum([parking_vars[i, j] * (4-j) for i in data['클러스터링'].unique() for j in range(4)])

# 제약 조건 설정
for i in data['클러스터링'].unique():
    for j in range(4):
        # 각 클러스터와 유동인구 레이블 조합별로 데이터 포인트의 수를 계산
        data_points_count = data[(data['클러스터링'] == i) & (data['유동인구+견인횟수'] == j)]['id'].count()
        # 헥사곤 5개당 최소 1개 주차장으로 비율 설정
        required_parking_spots = data_points_count / 5
        # 모델에 제약 조건 추가
        model += parking_vars[i, j] >= required_parking_spots



# 문제 해결
model.solve()

# 결과 출력
print("Status:", pulp.LpStatus[model.status])
for i in data['클러스터링'].unique():
    for j in range(4):
        print(f"Cluster {i}, Label {j}: Allocate {parking_vars[i, j].varValue} parking spots.")
'''

'\n# 문제 인스턴스 생성\nmodel = pulp.LpProblem("Enhanced_Parking_Allocation", pulp.LpMinimize)\n\n# 클러스터 및 레이블 조합별 변수 생성\nparking_vars = pulp.LpVariable.dicts("Park",\n                                     [(i, j) for i in data[\'클러스터링\'].unique() for j in range(4)],\n                                     lowBound=0, cat=\'Integer\')\n\n# 목표 함수 설정: 주차장 수 최소화, 레이블 가중치 반영\n    #(4-j)의 경우 레이블 등급에 따른 가중치 부여하는 과정(0등급일수록 높은 등급)\nmodel += pulp.lpSum([parking_vars[i, j] * (4-j) for i in data[\'클러스터링\'].unique() for j in range(4)])\n\n# 제약 조건 설정\nfor i in data[\'클러스터링\'].unique():\n    for j in range(4):\n        # 각 클러스터와 유동인구 레이블 조합별로 데이터 포인트의 수를 계산\n        data_points_count = data[(data[\'클러스터링\'] == i) & (data[\'유동인구+견인횟수\'] == j)][\'id\'].count()\n        # 헥사곤 5개당 최소 1개 주차장으로 비율 설정\n        required_parking_spots = data_points_count / 5\n        # 모델에 제약 조건 추가\n        model += parking_vars[i, j] >= required_parking_spots\n\n\n\n# 문제 해결\nmodel.solve()\n\n# 결과 출력\nprint("Status:", pulp.LpStatus[mo

In [6]:
# 문제 인스턴스 재생성
model = pulp.LpProblem("Balanced_Parking_Allocation", pulp.LpMinimize)

# 목표 함수와 변수 재설정: 등급 차이를 줄이는 가중치 조정
model += pulp.lpSum([parking_vars[i, j] * (1 + (7-j)/10) for i in data['클러스터링'].unique() for j in range(1, 8)])

# 제약 조건: 각 레이블에 최소 하나의 주차장 할당 보장
for i in data['클러스터링'].unique():
    for j in range(1, 8):
        data_points_count = data[(data['클러스터링'] == i) & (data['유동인구+견인횟수'] == j)].shape[0]
        required_parking_spots = max(1, data_points_count / 30)  # 30개 데이터 포인트 당 최소 1개 주차장
        model += parking_vars[i, j] >= required_parking_spots

        # 여기서 데이터 포인트는 특정 클러스터링 및 레이블 조합에 속하는 개별 관측치의 수를 의미
        # 예를 들어, 클러스터 1, 레이블 2의 데이터 포인트 수는 해당 조합에 속하는 모든 관측치의 총 수

# 최대 주차장 수 제약 유지
model += pulp.lpSum([parking_vars[i, j] for i in data['클러스터링'].unique() for j in range(1, 8)]) <= 300

# 문제 해결
model.solve()

# 결과 출력
print("Status:", pulp.LpStatus[model.status])
for i in data['클러스터링'].unique():
    for j in range(1, 8):
        print(f"Cluster {i}, Label {j}: Allocate {parking_vars[i, j].varValue} parking spots.")



NameError: name 'parking_vars' is not defined

## Set-Covering problem으로의 확장

In [ ]:
'''
# 각 클러스터별 상위 15개 위치 선정
top_locations_per_cluster = data.groupby('클러스터링').apply(lambda x: x.nlargest(15, 'score'))

# 위치 데이터 추출
positions = top_locations_per_cluster[['centroid_x', 'centroid_y']].to_numpy()

# 거리 행렬 계산
distance_matrix = cdist(positions, positions, metric='euclidean')

# 거리 임곗값
coverage_threshold_100m = 100  # 헥사곤 간의 거리를 반영해 100미터로 설정

# 커버리지 행렬 생성 (1이면 해당 위치가 다른 위치를 커버)
coverage_matrix_100m = (distance_matrix <= coverage_threshold_100m).astype(int)

# 선형 할당 문제로 최적화 (헝가리안 알고리즘 사용, -1을 곱해 최대 커버리지를 추구)
row_ind_100m, col_ind_100m = linear_sum_assignment(-coverage_matrix_100m)

# 최적 위치 선정 (중복 없이 최적 위치 인덱스 추출)
optimal_indices_100m = np.unique(col_ind_100m)
optimal_selection_100m = top_locations_per_cluster.iloc[optimal_indices_100m]

# 최적 위치의 상세 정보 출력
optimal_selection_100m[['id', 'centroid_x', 'centroid_y', 'kmeans_label', '유동인구', '견인횟수', 'score']]
'''

"\n# 각 클러스터별 상위 15개 위치 선정\ntop_locations_per_cluster = data.groupby('클러스터링').apply(lambda x: x.nlargest(15, 'score'))\n\n# 위치 데이터 추출\npositions = top_locations_per_cluster[['centroid_x', 'centroid_y']].to_numpy()\n\n# 거리 행렬 계산\ndistance_matrix = cdist(positions, positions, metric='euclidean')\n\n# 거리 임곗값\ncoverage_threshold_100m = 100  # 헥사곤 간의 거리를 반영해 100미터로 설정\n\n# 커버리지 행렬 생성 (1이면 해당 위치가 다른 위치를 커버)\ncoverage_matrix_100m = (distance_matrix <= coverage_threshold_100m).astype(int)\n\n# 선형 할당 문제로 최적화 (헝가리안 알고리즘 사용, -1을 곱해 최대 커버리지를 추구)\nrow_ind_100m, col_ind_100m = linear_sum_assignment(-coverage_matrix_100m)\n\n# 최적 위치 선정 (중복 없이 최적 위치 인덱스 추출)\noptimal_indices_100m = np.unique(col_ind_100m)\noptimal_selection_100m = top_locations_per_cluster.iloc[optimal_indices_100m]\n\n# 최적 위치의 상세 정보 출력\noptimal_selection_100m[['id', 'centroid_x', 'centroid_y', 'kmeans_label', '유동인구', '견인횟수', 'score']]\n"

In [ ]:
# 전체 할당된 주차장 수
total_spots = relevant_locations['allocated_parking_spots'].sum()

# 클러스터별 할당된 주차장 수
cluster_allocation_counts = relevant_locations.groupby('클러스터링')['allocated_parking_spots'].sum()

# 클러스터별 할당된 주차장 수 비율에 따라 각 클러스터에서 선택할 데이터 수 계산
cluster_select_counts = (cluster_allocation_counts / total_spots * 100).astype(int)

# 각 클러스터에서 비율에 맞게 상위 위치 선택
def select_top_by_cluster(df, cluster_counts):
    result = pd.DataFrame()
    for cluster, count in cluster_counts.items():
        top_spots = df[df['클러스터링'] == cluster].nlargest(count, 'allocated_parking_spots')
        result = pd.concat([result, top_spots], axis=0)
    return result

top_300_locations = select_top_by_cluster(relevant_locations, cluster_select_counts)

# 좌표 기반 거리 행렬 계산
positions = top_300_locations[['centroid_x', 'centroid_y']].to_numpy()
distance_matrix = cdist(positions, positions, metric='euclidean')

# 100미터 이내 커버리지 행렬 생성
coverage_threshold_100m = 100
coverage_matrix_100m = (distance_matrix <= coverage_threshold_100m).astype(int)

# 선형 할당 문제 해결
row_ind_100m, col_ind_100m = linear_sum_assignment(-coverage_matrix_100m)
optimal_indices_100m = np.unique(col_ind_100m)

# 최적 위치 선택
optimal_selection_100m = top_300_locations.iloc[optimal_indices_100m]

# 선택된 위치의 상세 정보 출력 및 저장
optimal_selection_details = optimal_selection_100m[['id', 'centroid_x', 'centroid_y', '클러스터링', '유동인구', '견인횟수', 'allocated_parking_spots']]
print(optimal_selection_details)
optimal_selection_details.to_csv('optimal_selection_100m.csv')


        id     centroid_x    centroid_y  클러스터링        유동인구  견인횟수  \
1     1575  963184.792317  1.949185e+06      0  323.728395   1.0   
10    1400  962924.984696  1.949635e+06      0    0.000000   3.0   
12    1401  962924.984696  1.949535e+06      0    0.000000   0.0   
14    1402  962924.984696  1.949435e+06      0    0.000000   0.0   
21    1344  962838.382156  1.949485e+06      0    0.000000   0.0   
...    ...            ...           ...    ...         ...   ...   
489   1614  963271.394858  1.951035e+06      2    0.000000   0.0   
1017  1623  963271.394858  1.950135e+06      3    0.000000   0.0   
1019  1617  963271.394858  1.950735e+06      3    0.000000   0.0   
1022  1628  963271.394858  1.949635e+06      3  479.462963   2.0   
1024  1624  963271.394858  1.950035e+06      3    0.000000   0.0   

      allocated_parking_spots  
1                        24.0  
10                       24.0  
12                       24.0  
14                       24.0  
21                     

In [ ]:
'''
# 모든 위치와 그에 할당된 주차장 수를 사용
def assign_spots(cluster_tow_pair, sub_df):
    # parking_vars에서 할당된 주차장 수 추출, 키가 없는 경우 0을 반환
    allocated_spots = parking_vars.get(cluster_tow_pair, pulp.LpVariable("default", lowBound=0, upBound=0)).varValue
    sub_df['allocated_parking_spots'] = allocated_spots
    return sub_df

# 각 클러스터와 유동인구+견인횟수 조합에 대해 함수 적용
all_locations_with_allocations = data.groupby(['클러스터링', '유동인구+견인횟수']).apply(lambda x: assign_spots((x.name[0], x.name[1]), x))

# 할당된 주차장 수가 0보다 큰 위치만 선택
relevant_locations = all_locations_with_allocations[all_locations_with_allocations['allocated_parking_spots'] > 0]


# 좌표 기반 거리 행렬 계산
from scipy.spatial.distance import cdist
positions = relevant_locations[['centroid_x', 'centroid_y']].to_numpy()
distance_matrix = cdist(positions, positions, metric='euclidean')

# 100미터 이내 커버리지 행렬 생성
coverage_threshold_100m = 100
coverage_matrix_100m = (distance_matrix <= coverage_threshold_100m).astype(int)

# 선형 할당 문제 해결
from scipy.optimize import linear_sum_assignment
row_ind_100m, col_ind_100m = linear_sum_assignment(-coverage_matrix_100m)
optimal_indices_100m = np.unique(col_ind_100m)

# 최적 위치 선택
optimal_selection_100m = relevant_locations.iloc[optimal_indices_100m]

# 선택된 위치의 상세 정보 출력 및 저장
optimal_selection_details = optimal_selection_100m[['id', 'centroid_x', 'centroid_y', '클러스터링', '유동인구', '견인횟수', 'allocated_parking_spots']]
print(optimal_selection_details)
optimal_selection_details.to_csv('optimal_selection_100m.csv')
'''

'\n# 모든 위치와 그에 할당된 주차장 수를 사용\ndef assign_spots(cluster_tow_pair, sub_df):\n    # parking_vars에서 할당된 주차장 수 추출, 키가 없는 경우 0을 반환\n    allocated_spots = parking_vars.get(cluster_tow_pair, pulp.LpVariable("default", lowBound=0, upBound=0)).varValue\n    sub_df[\'allocated_parking_spots\'] = allocated_spots\n    return sub_df\n\n# 각 클러스터와 유동인구+견인횟수 조합에 대해 함수 적용\nall_locations_with_allocations = data.groupby([\'클러스터링\', \'유동인구+견인횟수\']).apply(lambda x: assign_spots((x.name[0], x.name[1]), x))\n\n# 할당된 주차장 수가 0보다 큰 위치만 선택\nrelevant_locations = all_locations_with_allocations[all_locations_with_allocations[\'allocated_parking_spots\'] > 0]\n\n\n# 좌표 기반 거리 행렬 계산\nfrom scipy.spatial.distance import cdist\npositions = relevant_locations[[\'centroid_x\', \'centroid_y\']].to_numpy()\ndistance_matrix = cdist(positions, positions, metric=\'euclidean\')\n\n# 100미터 이내 커버리지 행렬 생성\ncoverage_threshold_100m = 100\ncoverage_matrix_100m = (distance_matrix <= coverage_threshold_100m).astype(int)\n\n# 선형 할당 문제 해결\

In [ ]:
'''
# 주차장 할당 결과에 따른 top 위치 선정
top_locations_per_cluster = data.groupby('클러스터링').apply(lambda x: x.nlargest(15, '할당된_주차장_수'))

# 좌표 기반 거리 행렬 계산
from scipy.spatial.distance import cdist
positions = top_locations_per_cluster[['centroid_x', 'centroid_y']].to_numpy()
distance_matrix = cdist(positions, positions, metric='euclidean')

# 100미터 이내 커버리지 행렬 생성
coverage_threshold_100m = 100
coverage_matrix_100m = (distance_matrix <= coverage_threshold_100m).astype(int)

# 선형 할당 문제 해결
from scipy.optimize import linear_sum_assignment
row_ind_100m, col_ind_100m = linear_sum_assignment(-coverage_matrix_100m)
optimal_indices_100m = np.unique(col_ind_100m)

# 최적 위치 선택
optimal_selection_100m = top_locations_per_cluster.iloc[optimal_indices_100m]

# 선택된 위치의 상세 정보 출력 및 저장
optimal_selection_details = optimal_selection_100m[['id', 'centroid_x', 'centroid_y', '클러스터링', '유동인구', '견인횟수', '할당된_주차장_수']]
print(optimal_selection_details)
optimal_selection_details.to_csv('optimal_selection_100m.csv')
'''

"\n# 주차장 할당 결과에 따른 top 위치 선정\ntop_locations_per_cluster = data.groupby('클러스터링').apply(lambda x: x.nlargest(15, '할당된_주차장_수'))\n\n# 좌표 기반 거리 행렬 계산\nfrom scipy.spatial.distance import cdist\npositions = top_locations_per_cluster[['centroid_x', 'centroid_y']].to_numpy()\ndistance_matrix = cdist(positions, positions, metric='euclidean')\n\n# 100미터 이내 커버리지 행렬 생성\ncoverage_threshold_100m = 100\ncoverage_matrix_100m = (distance_matrix <= coverage_threshold_100m).astype(int)\n\n# 선형 할당 문제 해결\nfrom scipy.optimize import linear_sum_assignment\nrow_ind_100m, col_ind_100m = linear_sum_assignment(-coverage_matrix_100m)\noptimal_indices_100m = np.unique(col_ind_100m)\n\n# 최적 위치 선택\noptimal_selection_100m = top_locations_per_cluster.iloc[optimal_indices_100m]\n\n# 선택된 위치의 상세 정보 출력 및 저장\noptimal_selection_details = optimal_selection_100m[['id', 'centroid_x', 'centroid_y', '클러스터링', '유동인구', '견인횟수', '할당된_주차장_수']]\nprint(optimal_selection_details)\noptimal_selection_details.to_csv('optimal_selection_100m.c

In [ ]:
# 원본 좌표 시스템 정의 (UTM 좌표)
original_crs = Proj(init='epsg:5179')  # UTM-K 좌표계 (한국)

# 변환할 좌표 시스템 정의 (WGS84, 위경도)
target_crs = Proj(init='epsg:4326')  # WGS84 좌표계 (위경도)

# 좌표 변환 함수
def convert_coordinates(x, y):
    lon, lat = transform(original_crs, target_crs, x, y)
    return lat, lon

# 'X'와 'Y' 좌표를 UTM-K 좌표계에서 WGS84 좌표계로 변환
optimal_selection_details['lon'], optimal_selection_details['lat'] = zip(*optimal_selection_details.apply(lambda row: convert_coordinates(row['centroid_x'], row['centroid_y']), axis=1))

# 변환된 위도(latitude)와 경도(longitude) 출력
print(optimal_selection_details[['lat', 'lon']])

/Users/donggunhan/Documents/Visual Studio Code/.venv/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

/Users/donggunhan/Documents/Visual Studio Code/.venv/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

/var/folders/d1/k8w7ydl912db_c31zskh1q700000gn/T/ipykernel_44387/140815338.py:9: FutureWarning:

This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1



             lat        lon
1     127.083263  37.541255
10    127.080300  37.545301
12    127.080305  37.544400
14    127.080310  37.543498
21    127.079327  37.543945
...          ...        ...
489   127.084151  37.557934
1017  127.084196  37.549822
1019  127.084166  37.555230
1022  127.084221  37.545315
1024  127.084201  37.548920

[98 rows x 2 columns]


/var/folders/d1/k8w7ydl912db_c31zskh1q700000gn/T/ipykernel_44387/140815338.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/d1/k8w7ydl912db_c31zskh1q700000gn/T/ipykernel_44387/140815338.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:

# 클러스터링 열을 기준으로 각 클러스터별로 개수 계산
cluster_counts = optimal_selection_details['클러스터링'].value_counts()

# 결과 출력
print("클러스터별 배치된 개수:")
print(cluster_counts)

클러스터별 배치된 개수:
0    51
1    39
2     4
3     4
Name: 클러스터링, dtype: int64


In [ ]:
optimal_selection_details.to_csv('optimal_selection_details.csv')